In [1]:
import os
import sys
import pandas as pd
import numpy as np
import regex as re
from matplotlib import pyplot as plt
import time

In [2]:
working_dir = "datasets/cancers/lung/"
os.chdir(working_dir)
dirs = os.listdir("data")

In [3]:
len(dirs)

1146

In [4]:
df_genes = pd.read_csv("https://stephenslab.github.io/count-clustering/project/utilities/gene_names_all_gtex.txt", header=None).set_index(0)
select_genes = df_genes.index.values

In [5]:
df = df_genes.copy()
#print(len(df['gene']))
#df = df.loc[select_genes,:]
#df=pd.read_csv("miRNA.txt")
df.head()

Empty DataFrame
Columns: []
Index: [ENSG00000000419, ENSG00000000457, ENSG00000000460, ENSG00000000938, ENSG00000000971]

In [6]:
len(df)

16069

In [7]:
#fpkm
def getFilenameFromDir(directory):
    for element in os.listdir(directory):
        if re.match("[a-zA-Z0-9]{8}-[a-zA-Z0-9]{4}-[a-zA-Z0-9]{4}-[a-zA-Z0-9\-]{4}-[a-zA-Z0-9\-]{12}[\.FPKM]{5}.txt[\.gz]{0,3}",element):
            cfile = element
            print(element)
            return cfile
    raise BaseException("Not found %s"%os.listdir(directory))

In [ ]:
#counts
def getFilenameFromDir(directory):
    for element in os.listdir(directory):
        if re.match("[a-zA-Z0-9]{8}-[a-zA-Z0-9]{4}-[a-zA-Z0-9]{4}-[a-zA-Z0-9\-]{4}-[a-zA-Z0-9\-]{12}[\.htseq]{6}[\.\_counts]{7}[\.txt]{0,4}[\.gz]{0,3}",element):
            cfile = element
            print(element)
            return cfile
    print("Nothing found")
    print(os.listdir(directory))

In [ ]:
#fpkm-uq
def getFilenameFromDir(directory):
    for element in os.listdir(directory):
        if re.match("[a-zA-Z0-9]{8}-[a-zA-Z0-9]{4}-[a-zA-Z0-9]{4}-[a-zA-Z0-9\-]{4}-[a-zA-Z0-9\-]{12}[\.FPKM-UQ]{0,8}[\.txt]{0,4}[\.gz]{0,3}",element):
            cfile = element
            print(element)
            return cfile

In [ ]:
##miRNA
def getFilenameFromDir(directory):
    for element in os.listdir(directory):
        if re.match("[a-zA-Z0-9]{8}-[a-zA-Z0-9]{4}-[a-zA-Z0-9]{4}-[a-zA-Z0-9\-]{4}-[a-zA-Z0-9\-]{12}[\.mirbase21\.mirnas\.quantification][\.txt]{0,4}[\.gz]{0,3}",element):
            cfile = element
            print(element)
            return cfile

In [8]:
maxacceptables = 15000
added = len(df.columns)
for i,cdirectory in enumerate(dirs):
    if re.match("manifest\.txt",cdirectory):
        print("SKIPPING %s "%cdirectory)
        continue
    if "Icon" in cdirectory:
        print("SKIPPING %s "%cdirectory)
        continue
    cfile = getFilenameFromDir("data/%s"%cdirectory)
    #put header=0 for miRNA!
    #cdf = pd.read_csv(("%s/data/%s/%s"%(working_dir,cdirectory,cfile)), sep='\t')
    #cdf.drop(labels=["reads_per_million_miRNA_mapped","cross-mapped"], axis=1, inplace=True) #miRNA only
    cdf = pd.read_csv(("data/%s/%s"%(cdirectory,cfile)), sep='\t', header=None)
    cdf.columns = ["gene", cfile[:]]
    cdf['gene'] = [gene[:15] for gene in cdf['gene']]
    cdf.set_index('gene',inplace=True)
    cdf = cdf.reindex(index=df_genes.index)
    old_L = len(df.columns)
    df.insert(0,cdf.keys()[0][:],cdf.values)
    if len(df.columns) != old_L+1:
        print(*sys.exc_info())
        raise(Exception("Not able to add: %s"%cfile))
    if added >= maxacceptables:
        break
print(added, i)

b1558748-bec9-4bea-a35b-040ca9a1f4cd.FPKM.txt.gz
6b1c72e9-ed03-4c02-91f6-cda07c1f9e49.FPKM.txt.gz
022cbd07-3e20-4144-b2ab-8c76adffae73.FPKM.txt.gz
85ac083c-dbb9-4541-8b75-d73ab216f54c.FPKM.txt.gz
779250be-012c-4e5b-9b4e-258192607c4b.FPKM.txt.gz
8dd3161b-a1d9-453c-afb8-293b20330f3d.FPKM.txt.gz
cf792f7f-a3ee-4544-b029-8c4e4da4f973.FPKM.txt.gz
186b83bd-ec3f-42b4-ba91-d86c9618d5e3.FPKM.txt.gz
ce786209-c77d-47e0-95a6-20c036bed8b3.FPKM.txt.gz
db0ce49b-0048-4ba2-9295-309d8c570bb4.FPKM.txt.gz
68acaa5e-6a27-499f-925d-710af9d30710.FPKM.txt.gz
808353e4-59a3-4df0-90fd-bd42b7af366e.FPKM.txt.gz
c8fb3217-2cc9-47c0-84b6-b666c07624e6.FPKM.txt.gz
ef6e0187-ca29-4c32-9731-4f9a6b51ff80.FPKM.txt.gz
7b4a2fcb-95eb-4cc2-8fde-ec0851ffb724.FPKM.txt.gz
1208db51-3816-4c68-8b01-b494b14063cf.FPKM.txt.gz
e4de8f87-b9c8-4021-b29a-b7d128ff64a3.FPKM.txt.gz
dc55b4b9-f3e6-4772-8421-26b04b750716.FPKM.txt.gz
e9205c80-39e9-446d-8151-f493e43228c4.FPKM.txt.gz
26918c74-1269-4f63-a871-57855b4aa846.FPKM.txt.gz
39423508-c376-4582-8

In [ ]:
#df = pd.read_csv(("%s/mainTable.csv"%working_dir))

In [10]:
print(("genes:%d\trealizations:%d"%(len(df.index),len(df.columns))))

genes:16069	realizations:1145


In [ ]:
df.dropna().round().astype(float)

In [11]:
df.dropna(how='all', axis=0).round(decimals=2).to_csv("mainTable_all.csv", index=True)

expression per gene

In [ ]:
np.sum(df.loc[0,:].values[1:])

expression per case

In [ ]:
np.sum(df.loc[:,df.keys()[1]].values)

In [ ]:
df_files=pd.read_csv("files.dat",header=0,index_col=0)
df_files[df_files.index.isin(df.columns)].to_csv("files.dat",index=True,header=True)